In [1]:
import os,sys,pymongo
import DSAHelperFunctions as dsa
from pprint import pprint

from openslide import OpenSlide
from bson.objectid import ObjectId
from flask import Response
from PIL import Image
import os, gridfs, cStringIO
from random import shuffle

db = pymongo.MongoClient('localhost',27017)['masterSlideList']
UberDBcon = db['RawSlideData']
gfs = gridfs.GridFS(db)

In [2]:
from IPython.parallel import Client
rc = Client()
dview = rc[:]
dview.block=False

/home/dgutman/anaconda/envs/dsa_girder/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [3]:
Config = {}
Config["db_name"] = "masterSlideList"
Config["db_collection"] = "RawSlideData"
Config["thumbSize"] = "small"
Config["thumb_small_height"] = 128

In [4]:
%px import gridfs, os,pymongo
%px import cStringIO
%px db = pymongo.MongoClient('localhost',27017)['masterSlideList']
%px UberDBcon = db['RawSlideData']
%px gfs = gridfs.GridFS(db)

%px config = {}
%px config["db_name"] = "masterSlideList"
%px config["db_collection"] = "RawSlideData"
%px config["thumbSize"] = "small"
%px config["thumb_small_height"] = 128
%px from openslide import OpenSlideError
%px from openslide import OpenSlide
# %px from . import curConfig

In [5]:
AllImageList = list(UberDBcon.find())

In [6]:
from DSAHelperFunctions import LinePrinter       

filesProc = needsThumb = hasThumb = 0
needsThumb_ImageList = []

for image in AllImageList:
    thumbSize = 'small'
    thumbPix = 128
    filesProc +=1
    sp = image['path']
    ## I am only going to set the width... not the height
    filename = os.path.splitext(os.path.basename(sp))[0] + "." + str(thumbSize) + ".jpg"
    if not gfs.exists(filename=filename):
        #print filename
        needsThumb +=1
        needsThumb_ImageList.append(image)
    else:
        hasThumb +=1
    genStatus = "Made %d thumbs and already had %d thumbs out puf %d files" % (needsThumb,hasThumb,filesProc)
    LinePrinter(genStatus)
        
print filesProc,needsThumb, hasThumb

Made 21454 thumbs and already had 18602 thumbs out puf 40056 files40056 21454 18602


In [7]:
# db_name: UberSliderSorter
# db_collection: RawSlideData
config = {}
config["db_name"] = "masterSlideList"
config["db_collection"] = "RawSlideData"
config["thumbSize"] = "small"
config["thumb_small_height"] = 128


@dview.parallel(block=False)
def CheckForThumbnail_V2(curSlideInfo):
    """ I am passing the entire data object from the RawSlideData collection
    which lists the path, and other file properties I want to process"""
    thumbSize = 128
    thumbSize = 'small'

    ##Keeping this consistent with Mohammed's Implementation in the API
    image = curSlideInfo
    print image
    path = image["path"]
    filename = os.path.splitext(os.path.basename(path))[0] + "." + str(thumbSize) + ".jpg"

    ### Need to make sure that we always use level0???
    if not gfs.exists(filename=filename):
        width = int(image["scanProperties"]["openslide_level[0]_width"])
        height = int(image["scanProperties"]["openslide_level[0]_height"])
        thumbHeight = float(config["thumb_small_height"])  
        thumbWidth = int(round(thumbHeight/float(height) * int(width)))
        infoString = "%d %d %d %d " % ( width,height,thumbHeight,thumbWidth )
        try:
            osr = OpenSlide(path)
            thumb = osr.get_thumbnail((thumbWidth,thumbHeight))
        except OpenSlideError, e:
            resp = {"status": 404, "message": "OpenSlideError: Thumbnail failed to load"}
            badImageBuf = open('/home/dgutman/devel/dsa_girdertools/error.128.jpg', contentType="image/jpeg", filename=filename, thumbCreated=False,errorThumb=True)
            return resp
        except ValueError:
            resp = {"status": 404, "message": "Thumbnail failed to load got ValueError"}
            badImageBuf = open('/home/dgutman/devel/dsa_girdertools/error.128.jpg', contentType="image/jpeg", filename=filename, thumbCreated=False,errorThumb=True)
            return resp
        except:
            resp = {"status": 404, "message": "Thumbnail failed to load- Not sure Why  %s" % (infoString)}
            badImageBuf = open('/home/dgutman/devel/dsa_girdertools/error.128.jpg', contentType="image/jpeg", filename=filename, thumbCreated=False,errorThumb=True)
            return resp
        buf = cStringIO.StringIO()
        thumb.save(buf, 'jpeg', quality=90)
        gfs.put(buf.getvalue(), contentType="image/jpeg", filename=filename, thumbCreated=True)
        return {"status": 200, "message": "Image Generated"}
    else:
 
        return {"status": 200, "message": "Image PreCached"}


# dgutman@digitalslidearchive:~/devel/dsa_girdertools$ convert error.png.1 error.128.jpg
# dgutman@digitalslidearchive:~/devel/dsa_girdertools$ file error.128.jpg
# error.128.jpg: JPEG image data, JFIF standard 1.01
# dgutman@digitalslidearchive:~/devel/dsa_girdertools$ pwd
# /home/dgutman/devel/dsa_girdertools

In [8]:
print len(needsThumb_ImageList)
shuffle(needsThumb_ImageList)###shuffle things to avoid hitting the same file system each time

ar = CheckForThumbnail_V2.map(needsThumb_ImageList)

21454


In [10]:
print ar.elapsed
print ar.done()

208.911962
False
